In [2]:
import json
import time
import pandas as pd
from seleniumwire.utils import decode
from seleniumwire import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def load():
    options = {'disable_encoding': True}

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome(ChromeDriverManager().install(),seleniumwire_options=options )

    # Go to the Google home page
    driver.get("https://food.grab.com/ph/en/")
    return driver

In [4]:
driver = load()



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/benai/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [15]:
driver

/var/folders/c5/2p16_3w12w3f3_l7r1l9m2800000gn/T/ipykernel_1094/4288699930.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("location-input").send_keys("pasay")


In [6]:
def load_more(driver):
    return driver.find_element_by_css_selector("button.ant-btn").click()

In [7]:
input_id = driver.find_element_by_id("location-input").send_keys("pasay")
load_more(driver)

/var/folders/c5/2p16_3w12w3f3_l7r1l9m2800000gn/T/ipykernel_1094/1415324294.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  input_id = driver.find_element_by_id("location-input").send_keys("pasay")
/var/folders/c5/2p16_3w12w3f3_l7r1l9m2800000gn/T/ipykernel_1094/2217644237.py:2: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  return driver.find_element_by_css_selector("button.ant-btn").click()


In [8]:
main_df = pd.DataFrame()

In [9]:
def get_body(request):
    response = request.response
    body_byte = decode(response.body, response.headers.get('Content-Encoding', 'identity'))
    body_dict = json.loads(body_byte.decode("utf-8"))
    return body_dict

In [10]:
def get_popular_restaurant_latlng(popular_restaurants):
    body = get_body(popular_restaurants)
    
    name = []
    lat = []
    lng = []
    
    for i in range(len(body['recommendedMerchantGroups'][0]['recommendedMerchants'])):
        lat.append(body['recommendedMerchantGroups'][0]['recommendedMerchants'][i]['latlng']['latitude'])
        lng.append(body['recommendedMerchantGroups'][0]['recommendedMerchants'][i]['latlng']['longitude'])
        name.append(body['recommendedMerchantGroups'][0]['recommendedMerchants'][i]['address']['name'])
    
    df = pd.DataFrame(list(zip(name, lat, lng)), columns=['Name','latitude','longitude'])
    return df

In [11]:
def get_restaurant_latlng(all_restaurants):
    res_body = get_body(all_restaurants)
    
    name = []
    lat = []
    lng = []
    
    for i in range(len(res_body['searchResult']['searchMerchants'])):
        lat.append(res_body['searchResult']['searchMerchants'][i]['latlng']['latitude'])
        lng.append(res_body['searchResult']['searchMerchants'][i]['latlng']['longitude'])
        name.append(res_body['searchResult']['searchMerchants'][i]['address']['name'])
    
    df = pd.DataFrame(list(zip(name, lat, lng)), columns=['Name','latitude','longitude'])
    return df

In [12]:
all_restaurants = driver.wait_for_request('/foodweb/v2/search')
popular_restaurants = driver.wait_for_request('https://portal.grab.com/foodweb/v2/recommended/merchants')

In [13]:
main_df = main_df.append(get_popular_restaurant_latlng(popular_restaurants))

In [14]:
main_df.reset_index(drop=True)

,Name,latitude,longitude
0,Jollibee - G. Tuazon-Balic Balic,14.606886,121.003053
1,McDonald's - Governor Forbes,14.612865,120.989831
2,Wendy's - Dapitan [Available for LONG-DISTANCE...,14.611880,120.988455
3,Chowking - Pureza,14.601538,121.004724
4,WOW Shawarma - Cristobal Street,14.612964,120.994894
5,KFC - Nagtahan,14.600148,120.999737
6,Greenwich - Kanlaon,14.618849,121.002357
7,Black Scoop Cafe - UST Dapitan-Dos Castillas [...,14.613180,120.990617


In [ ]:
has_more = True
while has_more == True:
    df = get_restaurant_latlng(all_restaurants)
    main_df = main_df.append(df)
    
    time.sleep(10)
    try:
        load_more(driver)
    except:
        has_more = False

In [345]:
main_df.to_csv('restaurant_latlng.csv')

In [ ]:
main_df.shape()

In [304]:
driver.quit()

In [16]:
def get_request_id(url):
    req_id = 0
    for logs in log:
        log_json = json.loads(logs['message'])
        if log_json['message']['method'] == 'Network.requestWillBeSentExtraInfo':
            if ':path' in log_json['message']['params']['headers'].keys():
                if log_json['message']['params']['headers'][':path'] == url:
                    req_id = log_json['message']['params']['requestId']
                    print("request id", req_id)
                    if len(req_id) < 10:
                        return req_id